# Notebook Purpose

This notebook serves as documented walkthrough for the creation of the `illinois-gambling` postgresql database.

### Data Sources

This database contains data from the Illinois Gaming Board's monthly reports for both [Video Gambling](https://www.igb.illinois.gov/VideoReports.aspx) and [Casino Gambling](https://www.igb.illinois.gov/CasinoReports.aspx) as well as demographic data for each Illinois municipality from the 5 year American Community Survey.

<center><img src="../static/schema.png" width="1000"></center>

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import os
import numpy as np
from geopy import geocoders  
import matplotlib.pyplot as plt
from io import BytesIO
from geopy.geocoders import Nominatim
import psycopg2
from requests import get
import zipfile
import censusdata
import json
from geopy.distance import geodesic
from sqlalchemy import create_engine
from tqdm.notebook import tqdm
geolocator = Nominatim(user_agent='Illinois Gambling')

# Create Database

Connect to postgresql

In [2]:
conn = psycopg2.connect(dbname="postgres")
cursor = conn.cursor()

Set configurations 

In [3]:
conn.autocommit = True 
conn.set_isolation_level(0)

If creating the database from scratch, uncomment the code below to drop the database if it exists and create an empty `illinois-gaming` database.

In [4]:
# cursor.execute('DROP DATABASE IF EXISTS illinois_gambling;')
# cursor.execute('CREATE DATABASE illinois_gambling;')
# conn.close()

Now that the database is created, open a connection to the `illinois-gambling`  database

In [5]:
conn = psycopg2.connect(dbname="illinois_gambling")
user = conn.get_dsn_parameters()['user']
port = conn.get_dsn_parameters()['port'] 
# Create sqlalchemy engine to streamline the sql process
engine = create_engine(f'postgresql+psycopg2://{user}:@localhost:{port}/illinois_gambling')

# Casino Table

In [6]:
with tqdm(total = 3, desc = ''.rjust(35, '–'), ncols=800) as pbar:
    pbar.desc = ' Reading scraped data...'.rjust(35, '–')
    # Create import path for the casino gambling data
    casino_path = os.path.join(os.pardir,'data', 'casino_gambling', 'casino_data.csv')
    # Read in csv file
    casino_table = pd.read_csv(casino_path)
    # Change date column from string to datetime
    casino_table['date'] = pd.to_datetime(casino_table.date)
    pbar.update(1)
    
# =======================================================================
    pbar.desc = ' Cleaning casino data...'.rjust(35, '–')
    # Data Cleaning: 
    # The naming of casinos is somewhat inconsistent.
    # Below, we map names that deviate from the majority
    # so they are consistent across observations
    CASINO_REFORMAT = {'Alton': 'ALTON - ARGOSY',
                       'Argosy Casino Alton': 'ALTON - ARGOSY',
                    'East Peoria': 'E. PEORIA - PAR-A-DICE', 
                       'Par-A-Dice Hotel Casino': 'E. PEORIA - PAR-A-DICE',
                    'Rock Island': 'ROCK ISLAND - JUMERS', 
                       "Jumer's Casino & Hotel": 'ROCK ISLAND - JUMERS',
                     'Joliet - Empress': 'JOLIET - Argosy Empress Casino', 
                     'Metropolis': 'METROPOLIS - HARRAHS',
                    "Joliet - Harrah's": 'JOLIET - HARRAHS',
                    "Harrah's Joliet Casino & Hotel": 'JOLIET - HARRAHS',
                    'Aurora': 'AURORA - HOLLYWOOD', 
                    'Hollywood Casino Aurora': 'AURORA - HOLLYWOOD',
                    'E  St  Louis': 'E. ST. LOUIS - CASINO QUEEN',
                    'E St  Louis': 'E. ST. LOUIS - CASINO QUEEN',
                    'DraftKings at Casino Queen': 'E. ST. LOUIS - CASINO QUEEN',
                    'Elgin': 'ELGIN - GRAND VICTORIA',
                    'Grand Victoria Casino': 'ELGIN - GRAND VICTORIA',
                    'Joliet - Hollywood': 'JOLIET - HOLLYWOOD',
                       'Hollywood Casino Joliet': 'JOLIET - HOLLYWOOD',
                    'Des Plaines': 'DES PLAINES - RIVERS CASINO', 
                       'Rivers Casino': 'DES PLAINES - RIVERS CASINO',
                    'METROPOLIS - HARRAHS*': 'METROPOLIS - HARRAHS',
                      "Harrah's Metropolis Casino": 'METROPOLIS - HARRAHS'}

    def casino_map(casino):
        if casino in CASINO_REFORMAT:
            return CASINO_REFORMAT[casino]
        else:
            return casino

    casino_table['casino'] = casino_table.casino.apply(casino_map)

    # Data Cleaning:
    # To make the columns more informative, we seperate the 
    # name of the casino from the name of the municipality.
    casino_table['municipality'] = casino_table.casino.apply(lambda x: x.split(' - ')[0])
    
#     def find_error(text):
#         try:
#             return text.split(' - ')[1]
#         except:
#             print(text)
#             ValueError('Oops')
            
    casino_table.casino.apply(find_error)
    # casino_table['casino'] = casino_table.casino.apply(lambda x: x.split(' - ')[1])
    casino_table['municipality'] = casino_table.municipality.apply(lambda x: x.title().strip().replace('E.', 'East'))
# =======================================================================
    pbar.desc = ' Searching for geo data'.rjust(35, '–')
    # The location of the casino is useful information
    # for measuring the relationship between video gambling
    # and casino performance. The addresses were searched manually 
    # Using a google search. While it would be valuable to have a 
    # dynamic way of fetching this information, it is unlikely that
    # these data points will change. 
    casino_addresses = {'Alton': '1 Piasa St, Alton, IL 62002',
    'East Peoria':'21 Blackjack Blvd, East Peoria, IL 61611',
    'Rock Island':'777 Jumer Dr, Rock Island, IL 61201',
    'Joliet': '777 Hollywood Blvd, Joliet, IL 60436',
    'Aurora':'100 E Front St, Metropolis, IL 62960',
    'East St. Louis': '200 S Front St, East St Louis, IL 62201',
    'Elgin': '250 S Grove Ave, Elgin, IL 60120',
    'Des Plaines':'3000 S River Rd, Des Plaines, IL 60018',
    'Metropolis': '100 E Front St, Metropolis, IL 62960'}

    casino_table['address'] = casino_table.municipality.map(casino_addresses)

    # The two columns below are aggregates of other columns. Because these numbers are 
    # easily calculable from the existing data, they are dropped. 
    # casino_table.drop(['agr_per_square_foot', 'agr_per_admission'], axis = 1, inplace=True)

    def coordinates_from_address(address):

        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude

    coordinates_from_address(casino_table.address[0])

    casino_coordinates = {}
    for address in casino_table.address.unique():
        try:
            latitude, longitude = coordinates_from_address(address)
            casino_coordinates[address] = (latitude, longitude)
        except:
            continue
    pbar.desc = ' Matching coordinates...'.rjust(35, '–')
    casino_coordinates['777 Hollywood Blvd, Joliet, IL 60436'] = (41.479289, -88.145328)
    casino_table['latitude'] = casino_table.address.apply(lambda x: casino_coordinates[x][0])
    casino_table['longitude'] = casino_table.address.apply(lambda x: casino_coordinates[x][1])


    casino_table.rename({'casino': 'name', 'agr': 'adjusted_gross_receipt',
                        'casino_square_feet': 'square_feet'}, axis = 1, inplace=True)
    pbar.update(1)
# =======================================================================
    pbar.desc = ' Inserting data into casino table...'.rjust(35, '–')
    casino_table = casino_table[['address', 'name', 'date', 'municipality',
                  'adjusted_gross_receipt', 'square_feet', 'admissions',
                   'state_share', 'local_share','latitude', 'longitude']]

    casino_table.to_sql('casino', engine, if_exists='append', index=True)
    pbar.desc = ' Complete'.rjust(35, '–')
    pbar.update(1)
# =======================================================================

# Video Gambling Table

In [7]:
with tqdm(total=3, desc = ''.rjust(31, '–'), ncols=800) as pbar:
    pbar.desc = ' Reading scraped data...'.rjust(31, '–')
    data_path = os.path.join(os.pardir,'data', 'video_gambling')
    file_path = [file for file in os.listdir(data_path) if file.endswith('.csv')][0]
    video_gambling_path = os.path.join(data_path, file_path )
    video_gambling_table = pd.read_csv(video_gambling_path)
    pbar.update(1)
    pbar.desc = ' Cleaning video gambling data...'.rjust(31)
    video_gambling_table['date'] = pd.to_datetime(video_gambling_table.date, format='%B-%Y')
    video_gambling_table = video_gambling_table.dropna()

    video_gambling_table = video_gambling_table[['date','Establishment', 'License Number', 'Municipality', 'VGT Count',
           'Amount Played', 'Amount Won', 'Net Wager', 'Funds In', 'Funds Out',
           'Net Terminal Income', 'NTI Tax', 'State Share', 'Municipality Share'
           ]]

    column_edit = lambda x: x.lower().strip().replace(' ', '_').replace('-', '_')
    video_gambling_table.columns = [column_edit(column) for column in video_gambling_table.columns]
    video_gambling_table.rename({'vgt_count': 'terminal_count'}, axis = 1, inplace=True)
    pbar.update(1)
    pbar.desc = ' Inserting table into database...'.rjust(31, '–')
    video_gambling_table.to_sql('video_gambling', engine, if_exists='append', index=True)
    pbar.desc = ' Complete!'.rjust(31, '–')
    pbar.update(1)

# Municipality Table

In [11]:
with tqdm(total=6, desc = ''.rjust(31), ncols=800) as pbar:
    # ============================================================================
    # Importing dataset for tracking whether or not a municipality has
    # prohibited video gambling
    prohibition = pd.read_csv('../data/MunicipalityList.csv')
    prohibition.dropna(subset=['Name'],inplace=True)
    # ============================================================================
    # Request census geo id dataset
    pbar.desc = ' Requesting Census Geo Ids'.rjust(39, '–')
    response = get('https://www2.census.gov/programs-surveys/acs/summary_file/2018/data/5_year_by_state/Illinois_All_Geographies_Not_Tracts_Block_Groups.zip')
    pbar.desc = ' Reading response Bytes'.rjust(39, '–')
    translated = BytesIO(response.content)
    pbar.desc = ' Unzipping census data'.rjust(39)
    zip_file = zipfile.ZipFile(translated)
    csv_raw = zip_file.open('g20185il.csv')
    pbar.desc = ' Reading id file'.rjust(39, '–')
    municipality_ids = pd.read_csv(csv_raw, header=None)
    municipality_ids = municipality_ids[municipality_ids.iloc[:,2] == 160]
    pbar.update(1)
    # ============================================================================
    # Collect ids for municipalities
    pbar.desc = ' Filtering for municipalities'.rjust(39, '–')
    census_ids = {}
    counties = video_gambling_table[video_gambling_table.municipality.str.contains('county', case=False)].index
    municipalities = video_gambling_table.loc[[x for x in video_gambling_table.index if x not in counties]]
    towns_unique = list(municipalities.municipality.unique())
    towns_unique += [town for town in casino_table.municipality.unique() if town not in towns_unique]
    towns_unique += [town for town in prohibition.Name.unique() if town not in towns_unique]
    # ============================================================================
    # Collecting Census Data
    pbar.desc = ' Matching Census ID to Municipality Name'.rjust(39, '–')
    for town in towns_unique:
        try:
            search = municipality_ids[municipality_ids.iloc[:,49].str.contains(town, case=False)]
            search  = search[~search.iloc[:,49].str.contains('CDP')]
            id_ = search.iloc[:,48].values[0]
            census_ids[town] = id_
        except:
            census_ids[town] = None
    pbar.desc = ' Addressing alternate string formats'.rjust(39, '–')
    error_cities = {'La Salle': 'lasalle',
     'Lamoille': 'la moille',
     'Windsor (Mercer)': 'windsor village' ,
     'Windsor (Shelby)': 'windsor city',
     'Leroy': 'le roy',
     'Depue': 'de pue',
     'Wilmington (Will)': 'wilmington city',
     'Sainte Marie': 'ste. marie',
     'Gulfport': 'Gulf port',
     'Saint Elmo': 'St. Elmo',
     'Whiteash': 'Whiteash',
     'Wilmington (Greene)': 'wilmington village',
     'Garden Prairie': 'Garden Prairie'}

    for city in error_cities:
        id_ = municipality_ids[municipality_ids.iloc[:,49].str.contains(error_cities[city], case=False)].iloc[:,48].values[0]
        census_ids[city] = id_
    pbar.desc = ' Finalizing municipal IDs'.rjust(39, '–')
    municipal_table = pd.DataFrame()
    municipal_table['name'] = towns_unique
    municipal_table['id'] = municipal_table.name.map(census_ids)
    municipal_table.dropna(subset=['id'], inplace=True)
    pbar.update(1)
    # ============================================================================
    pbar.desc = ' Collecting coordinates'.rjust(39, '–')
    def find_geo_data(municipality):

        location = geolocator.geocode(f'{municipality}, Illinois')
        if location:
            return location.raw

    town_geo = {}

    for town in towns_unique:
        town_geo[town] = find_geo_data(town)

    def collect_coordinates(town):
        return town_geo[town]['lat'], town_geo[town]['lon'], town_geo[town]['boundingbox']
    
    pbar.desc = ' Finalizing coordinates'.rjust(39, '–')
    municipal_table['latitude'], municipal_table['longitude'], municipal_table['boundingbox'] = zip(*municipal_table.name.apply(collect_coordinates))
    pbar.update(1)
    # ============================================================================
    pbar.desc = ' Collecting county data'.rjust(39, '–')
    def county_info(row):
        county_json = get(f'https://geo.fcc.gov/api/census/area?lat={row.latitude}&lon={row.longitude}&format=json').json()['results'][0]
        county_fips = county_json['county_fips']
        county_name = county_json['county_name']
        return county_fips, county_name

    municipal_table['county_fips'] = ''
    municipal_table['county_name'] = ''
    for idx, row in municipal_table.iterrows():
        county_fips, county_name = county_info(row)
        municipal_table.at[idx, 'county_fips'] = county_fips
        municipal_table.at[idx, 'county_name'] = county_name
    pbar.update(1)
    # ============================================================================
    pbar.desc = ' Identifying nearest casino'.rjust(39, '–')
    closest_casinos = []
    distance_from_casino = []
    for idx, row in municipal_table.iterrows():
        town_coor = (row.latitude, row.longitude)
        distances = {}
        for casino in casino_coordinates:
            casino_coor = casino_coordinates[casino]
            miles = geodesic(town_coor, casino_coor).miles
            distances[casino] = miles
            # ADD DISTANC INFO
        sort = sorted(distances.items(), key=lambda x: x[1])[0][0]
        closest_casinos.append(sort)

    municipal_table['nearest_casino'] = closest_casinos
    pbar.update(1)
    # ============================================================================
    pbar.desc = 'Inserting table into database'.rjust(39, '–')
    municipal_table = municipal_table[['name', 'id', 'nearest_casino','latitude', 'longitude', 'boundingbox', 'county_fips', 'county_name']]
    municipal_table.rename({'id': 'geo_id'}, axis=1, inplace=True)
    municipal_table.to_sql('municipality', engine, if_exists='replace', index=False)
    pbar.desc = ' Complete!'.rjust(39)
    pbar.update(1)

/Users/joel/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3,32,33,34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/joel/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [12]:
munic = pd.read_sql('''select * from municipality;''', engine)
census_ids = munic[['name', 'geo_id']].set_index('name').to_dict()['geo_id']



In [13]:
munic

,name,geo_id,nearest_casino,latitude,longitude,boundingbox,county_fips,county_name
0,Hoopeston,16000US1736061,"777 Hollywood Blvd, Joliet, IL 60436",40.4672587,-87.6683573,"(40.437915,40.477669,-87.689353,-87.640314)",17183,Vermilion
1,Centralia,16000US1712164,"200 S Front St, East St Louis, IL 62201",38.5250491,-89.1334037,"(38.463629,38.548746,-89.175645,-89.063264)",17121,Marion
2,Carpentersville,16000US1711358,"250 S Grove Ave, Elgin, IL 60120",42.1211364,-88.2578582,"(42.102677,42.1487296,-88.3593718,-88.2413703)",17089,Kane
3,Flora,16000US1726454,"200 S Front St, East St Louis, IL 62201",38.6689364,-88.4856042,"(38.655188,38.685662,-88.508657,-88.440763)",17025,Clay
4,Staunton,16000US1772403,"1 Piasa St, Alton, IL 62002",39.0122684,-89.7912063,"(38.998353,39.029017,-89.82316,-89.746843)",17117,Macoupin
...,...,...,...,...,...,...,...,...
1294,Windsor,16000US1782309,"777 Jumer Dr, Rock Island, IL 61201",41.2022584,-90.4423521,"(41.194901,41.207064,-90.45109,-90.436603)",17131,Mercer
1295,Winfield,16000US1782400,"250 S Grove Ave, Elgin, IL 60120",41.8616956,-88.1609031,"(41.8540185,41.901662,-88.179496,-88.124587)",17043,DuPage
1296,Winnetka,16000US1782530,"3000 S River Rd, Des Plaines, IL 60018",42.1080703,-87.7365286,"(42.0891766,42.1283745,-87.7707407,-87.7107537)",17031,Cook
1297,Woodland,16000US1783102,"777 Hollywood Blvd, Joliet, IL 60436",40.7144803,-87.7311426,"(40.707172,40.728843,-87.739566,-87.725096)",17075,Iroquois


# Census Data

In [21]:
variables_path = os.path.join(os.pardir,'data', 'census_variables.json')
with open(variables_path, 'r') as file:
    variables = json.load(file)

### Add variable lookup table

In [22]:
lookup = pd.DataFrame(variables, index=[0]).T.reset_index()
lookup.columns = ['variable', 'description']
lookup['variable'] = lookup.variable.apply(lambda x: x.upper())

lookup.to_sql('demographics_lookup', engine, if_exists='replace')



### Create Demographics Table

In [23]:
variable_names = list(variables)
variable_names.reverse()
count = 0
errors = {}
data = pd.DataFrame()
for year in range(2012, 2019):
    with tqdm(total=len(census_ids)-1, desc=str(year)) as pbar:
        for idx in range(len(census_ids)):
            town = list(census_ids)[idx]
            try:
                id_ = census_ids[town][-5:]
                if id_ == '67158':
                    continue
                geo = censusdata.censusgeo([('state', '17'), ('place', id_)])
                frame = censusdata.download('acs5', year, geo, variable_names, key='ef2b118b032f366e377ba482a7e9cdbc8cbfd617')
                frame['geo_id'] = census_ids[town]
                frame['municipality'] = town
                frame['year'] = year
                frame = frame[['geo_id','municipality', 'year'] + variable_names]
                frame.index = [count]
                data = data.append(frame)
                count +=1 
                pbar.update(1)
            except:
                pbar.write(f'Error thrown for year {year}')
                pbar.update(1)
                continue

In [26]:
data.to_sql('demographics', engine, if_exists='replace')

print('Database created successfully!')

Database created successfully!
